# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 8 2023 8:30AM,256862,10,MSP218311,"Methapharm, Inc.",Released
1,Feb 8 2023 8:30AM,256862,10,MSP218312,"Methapharm, Inc.",Released
2,Feb 8 2023 8:30AM,256862,10,MSP218313,"Methapharm, Inc.",Released
3,Feb 8 2023 8:30AM,256862,10,MSP218315,"Methapharm, Inc.",Released
4,Feb 8 2023 8:30AM,256861,10,MSP218317,"Methapharm, Inc.",Released
5,Feb 8 2023 8:30AM,256861,10,MSP218318,"Methapharm, Inc.",Released
6,Feb 8 2023 8:30AM,256862,10,MSP218319,"Methapharm, Inc.",Released
7,Feb 8 2023 8:30AM,256861,10,MSP218320,"Methapharm, Inc.",Released
8,Feb 8 2023 8:30AM,256862,10,MSP218321,"Methapharm, Inc.",Released
9,Feb 8 2023 8:30AM,256862,10,MSP218322,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
47,256868,Released,27
48,256870,Released,1
49,256872,Released,1
50,256873,Released,1
51,256875,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256868,NaN,2.0,27.0
256870,NaN,NaN,1.0
256872,NaN,NaN,1.0
256873,NaN,NaN,1.0
256875,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256777,0.0,0.0,1.0
256782,81.0,3.0,21.0
256783,0.0,0.0,52.0
256788,0.0,0.0,1.0
256797,0.0,0.0,38.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256777,0,0,1
256782,81,3,21
256783,0,0,52
256788,0,0,1
256797,0,0,38


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256777,0,0,1
1,256782,81,3,21
2,256783,0,0,52
3,256788,0,0,1
4,256797,0,0,38


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256777,,,1
1,256782,81,3,21
2,256783,,,52
3,256788,,,1
4,256797,,,38


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 8 2023 8:30AM,256862,10,"Methapharm, Inc."
4,Feb 8 2023 8:30AM,256861,10,"Methapharm, Inc."
13,Feb 8 2023 8:30AM,256875,10,ISDIN Corporation
14,Feb 8 2023 8:30AM,256865,10,ISDIN Corporation
19,Feb 8 2023 8:30AM,256856,10,ISDIN Corporation
32,Feb 8 2023 8:30AM,256857,10,Bio-PRF
43,Feb 7 2023 4:33PM,256783,15,"Brookfield Pharmaceuticals, LLC"
95,Feb 7 2023 4:31PM,256873,16,Sartorius Stedim Filters Inc.
96,Feb 7 2023 3:52PM,256870,21,"NBTY Global, Inc."
97,Feb 7 2023 3:48PM,256868,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 8 2023 8:30AM,256862,10,"Methapharm, Inc.",,,9
1,Feb 8 2023 8:30AM,256861,10,"Methapharm, Inc.",,,4
2,Feb 8 2023 8:30AM,256875,10,ISDIN Corporation,,,1
3,Feb 8 2023 8:30AM,256865,10,ISDIN Corporation,,,14
4,Feb 8 2023 8:30AM,256856,10,ISDIN Corporation,,,4
5,Feb 8 2023 8:30AM,256857,10,Bio-PRF,,,11
6,Feb 7 2023 4:33PM,256783,15,"Brookfield Pharmaceuticals, LLC",,,52
7,Feb 7 2023 4:31PM,256873,16,Sartorius Stedim Filters Inc.,,,1
8,Feb 7 2023 3:52PM,256870,21,"NBTY Global, Inc.",,,1
9,Feb 7 2023 3:48PM,256868,19,"AdvaGen Pharma, Ltd",,2,27


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 8:30AM,256862,10,"Methapharm, Inc.",9,,
1,Feb 8 2023 8:30AM,256861,10,"Methapharm, Inc.",4,,
2,Feb 8 2023 8:30AM,256875,10,ISDIN Corporation,1,,
3,Feb 8 2023 8:30AM,256865,10,ISDIN Corporation,14,,
4,Feb 8 2023 8:30AM,256856,10,ISDIN Corporation,4,,
5,Feb 8 2023 8:30AM,256857,10,Bio-PRF,11,,
6,Feb 7 2023 4:33PM,256783,15,"Brookfield Pharmaceuticals, LLC",52,,
7,Feb 7 2023 4:31PM,256873,16,Sartorius Stedim Filters Inc.,1,,
8,Feb 7 2023 3:52PM,256870,21,"NBTY Global, Inc.",1,,
9,Feb 7 2023 3:48PM,256868,19,"AdvaGen Pharma, Ltd",27,2,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 8:30AM,256862,10,"Methapharm, Inc.",9,,
1,Feb 8 2023 8:30AM,256861,10,"Methapharm, Inc.",4,,
2,Feb 8 2023 8:30AM,256875,10,ISDIN Corporation,1,,
3,Feb 8 2023 8:30AM,256865,10,ISDIN Corporation,14,,
4,Feb 8 2023 8:30AM,256856,10,ISDIN Corporation,4,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 8:30AM,256862,10,"Methapharm, Inc.",9.0,NaN,NaN
1,Feb 8 2023 8:30AM,256861,10,"Methapharm, Inc.",4.0,NaN,NaN
2,Feb 8 2023 8:30AM,256875,10,ISDIN Corporation,1.0,NaN,NaN
3,Feb 8 2023 8:30AM,256865,10,ISDIN Corporation,14.0,NaN,NaN
4,Feb 8 2023 8:30AM,256856,10,ISDIN Corporation,4.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 8:30AM,256862,10,"Methapharm, Inc.",9.0,0.0,0.0
1,Feb 8 2023 8:30AM,256861,10,"Methapharm, Inc.",4.0,0.0,0.0
2,Feb 8 2023 8:30AM,256875,10,ISDIN Corporation,1.0,0.0,0.0
3,Feb 8 2023 8:30AM,256865,10,ISDIN Corporation,14.0,0.0,0.0
4,Feb 8 2023 8:30AM,256856,10,ISDIN Corporation,4.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,6164037,182.0,13.0,0.0
12,256841,0.0,1.0,3.0
15,770428,73.0,4.0,81.0
16,770512,2.0,1.0,0.0
19,1027378,31.0,10.0,0.0
20,513613,2.0,0.0,0.0
21,1284170,5.0,0.0,0.0
22,256872,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,6164037,182.0,13.0,0.0
1,12,256841,0.0,1.0,3.0
2,15,770428,73.0,4.0,81.0
3,16,770512,2.0,1.0,0.0
4,19,1027378,31.0,10.0,0.0
5,20,513613,2.0,0.0,0.0
6,21,1284170,5.0,0.0,0.0
7,22,256872,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,182.0,13.0,0.0
1,12,0.0,1.0,3.0
2,15,73.0,4.0,81.0
3,16,2.0,1.0,0.0
4,19,31.0,10.0,0.0
5,20,2.0,0.0,0.0
6,21,5.0,0.0,0.0
7,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,182.0
1,12,Released,0.0
2,15,Released,73.0
3,16,Released,2.0
4,19,Released,31.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Completed,0.0,3.0,81.0,0.0,0.0,0.0,0.0,0.0
Executing,13.0,1.0,4.0,1.0,10.0,0.0,0.0,0.0
Released,182.0,0.0,73.0,2.0,31.0,2.0,5.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Completed,0.0,3.0,81.0,0.0,0.0,0.0,0.0,0.0
1,Executing,13.0,1.0,4.0,1.0,10.0,0.0,0.0,0.0
2,Released,182.0,0.0,73.0,2.0,31.0,2.0,5.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Completed,0.0,3.0,81.0,0.0,0.0,0.0,0.0,0.0
1,Executing,13.0,1.0,4.0,1.0,10.0,0.0,0.0,0.0
2,Released,182.0,0.0,73.0,2.0,31.0,2.0,5.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()